In [ ]:
from sklearn.metrics.pairwise import rbf_kernel
import numpy as np

def embed(sigma_vec, k_vec, X, Y, Gamma, kernel=True):
    def my_laplacian(A):
        D = A.sum(dim=0)
        return torch.diag(D) - A
    
    dist_X = torch.cdist(X, X)
    dist_Y = torch.cdist(Y, Y).cuda()
    n = len(Y)
    I_X = torch.eye(n).cuda()
    J = I_X - 1/n * torch.ones(n, n).cuda() #1/n_concat * torch.ones(n_concat, n_concat)
    WX = -0.5 * J @ dist_X @ J
    WY = -0.5 * J @ dist_Y @ J

    embeddings = {}
    rProjs = {}
    
    for k in k_vec:
        for sigma in sigma_vec:
            LY = my_laplacian(WY)
            if not kernel:
                LX = WX
            else: 
                gamma = 1/(sigma**2)
                w = rbf_kernel(X.cpu().numpy(), gamma=gamma)
                np.fill_diagonal(w, 0)
                LX = my_laplacian(torch.tensor(w)).cuda()
                
            diag_LX = torch.diag(0.5 * (1/(torch.diag(LX) + 0.000000001)))
            
            pLI = k*LY - LX
            embedding = (diag_LX @ pLI @ Gamma) + Gamma
            embeddings[(k, sigma)] = embedding
    
    return embeddings

In [ ]:
sigma_vec = [100]
k_vec = [1000]
dim = 2
Gamma = torch.randn(len(label_concat), dim).cuda()
embedding = embed(sigma_vec, k_vec, feature_concat, label_concat, Gamma)[(1000,100)]